In [ ]:
import rasterio
import cupy as cp
import cuspatial

from pathlib import Path

## Intro
This notebook takes Sentinel-2 JPEG2000 images and performs a simple GPU polygonize using cuPy and cuSpatial. Once vectorized the raster image can be used like any other vector dataset, and gain GPU acceleration from other RAPIDS libraries like cuDF and cuML.

## Sentinel 2 Data  
Sentinel-2 Data is open and free, it can be accessed here: https://scihub.copernicus.eu/dhus/#/home

The Sentinel-2 mission delivers medium/high-resolution optical images (from visible to shortwave infrared range) and frequent revisit times (every 5 days with two satellites in operation). Each tile covers 100km x 100km and is projected in UTM. The data is delivered as a set of tiles in JPEG2000 format.

In [ ]:
# TODO: upload this data to S3 or similar so we don't need to use Copernicus (it requires an account)
# TODO: I wonder if I can use NVJPEG2000's python bindings from this repo https://github.com/louis-she/nvjpeg2k-python to read the jp2s - could be a totally GPU solution
sentinel_2_data = r'S2A_MSIL2A_20230711T153821_N0509_R011_T18TWL_20230711T233752/S2A_MSIL2A_20230711T153821_N0509_R011_T18TWL_20230711T233752.SAFE/GRANULE/L2A_T18TWL_A042050_20230711T154201/IMG_DATA/R10m/'
sentinel_2_path = Path(sentinel_2_data)
band_jp2s = [x for x in sentinel_2_path.iterdir() if '_B0' in str(x)]

In [ ]:
### CPU-version - needed a 64GB swapfile on 32GB system memory; gave up after 33m runtime on an i9-13900k
# import geopandas as gpd
# import rasterio
# from shapely.geometry import shape

# # read the data and create the shapes 
# with rasterio.open(band_jp2s[0]) as f:
#     data = f.read(1)
#     data = data.astype('int16')
#     shapes = rasterio.features.shapes(data, transform=f.transform)

# # read the shapes as separate lists
# pixel_values = []
# geometry = []
# for shapedict, value in shapes:
#     pixel_values.append(value)
#     geometry.append(shape(shapedict))

# # build the gdf object over the two lists
# gdf = gpd.GeoDataFrame(
#     {'pixel_values': pixel_values, 'geometry': geometry },
#     crs="EPSG:32618"
# )



In [ ]:
def gpu_simple_polygonize(raster_file, bounding_box_mask=None):
    with rasterio.open(raster_file) as src:
        # Get the coordinates of the pixels
        x_coords = cp.arange(src.bounds.left, src.bounds.right, src.transform[0])
        y_coords = cp.arange(src.bounds.top + src.transform[4], src.bounds.bottom + src.transform[4], src.transform[4])        
    
    # TODO: Add cuProj Transformation to WGS here
    # TODO: Support masking to help reduce memory errors, this image covers a lot outside of NYC
    cols, rows = cp.meshgrid(x_coords, y_coords)
    del x_coords, y_coords

    # Create arrays representing the four corners of each polygon
    top_left = cp.dstack((cols, rows))
    top_right = cp.dstack((cols + src.transform[0], rows))
    bottom_right = cp.dstack((cols + src.transform[0], rows - src.transform[4]))
    bottom_left = cp.dstack((cols, rows - src.transform[4]))

    polygons = cp.stack((top_left, top_right, bottom_right, bottom_left, top_left), axis=2)
    del top_left, top_right, bottom_right, bottom_left, cols, rows

    interleaved_polygons = polygons.reshape(polygons.shape[0], polygons.shape[1], -1)
    del polygons

    return interleaved_polygons.flatten()


In [ ]:
# Generate the geometry from the raster
polygons_for_cuspatial = gpu_simple_polygonize(band_jp2s[0])
geometry_offsets = cp.arange(0, int(len(polygons_for_cuspatial)/10) + 1, 1)
part_offsets = geometry_offsets
ring_offsets = cp.arange(0, int(len(polygons_for_cuspatial)/2) + 1, 5)

cuspatial_geoseries = cuspatial.GeoSeries.from_polygons_xy(polygons_for_cuspatial, ring_offsets, part_offsets, geometry_offsets)
polygonized_raster = cuspatial.GeoDataFrame()
polygonized_raster['geometry'] = cuspatial_geoseries
del cuspatial_geoseries, polygons_for_cuspatial, geometry_offsets, part_offsets, ring_offsets

In [ ]:
# Add band values to the GeoDataFrame
for band in band_jp2s:
    band_number = str(band).split('_')[-2]
    with rasterio.open(band) as src:
        # Read pixel values and add to our cudf DataFrame (they return as a numpy array)
        polygonized_raster[band_number] = src.read().flatten()

In [ ]:
# Generate NDVI - a vegetation index, values over 0.5 are very likely to be vegetation
polygonized_raster['NDVI'] = (polygonized_raster['B08'] - polygonized_raster['B04']) / (polygonized_raster['B08'] + polygonized_raster['B04'])
vegetation = polygonized_raster[(polygonized_raster['NDVI'] >= 0.5) & (polygonized_raster['NDVI'] < 1)]
vegetation.reset_index(inplace=True, drop=True)

In [ ]:
# I wonder if we can do a DBSCAN that clusters based on band values that gives us a cluster similar to our vegetation pixels
# Not working right now, probably need to use cuml.preprocessing.StandardScaler to scale the data first
import cudf
from cuml.cluster import DBSCAN

polygonized_raster = polygonized_raster[:100000] # Sliced for memory reasons

dbscan = DBSCAN(eps=250, min_samples=2)
# Workaround for it breaking on the geometry column, hopefully a better way exists for memory reasons
clusters = dbscan.fit_predict(cudf.DataFrame([polygonized_raster.B02, polygonized_raster.B03, polygonized_raster.B04, polygonized_raster.B08]).astype('float32'))
clusters